# Grain Storage Analysis
### Overview 

***Environment*** 
 
**Operating System:** Pop!_OS  
**IDE:** VS Code with Jupyter extension  
**Python Version:** 3.10.12  

## Package Imports 

In [ ]:
# Data manipulation and analysis
import pandas as pd
import numpy as np

# File handling
import os
import openpyxl

# Machine learning
import sklearn

# Data visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Display options
pd.set_option('display.max_columns', None)

print("Environment setup complete!")

Environment setup complete!
Required packages imported: pandas, numpy, openpyxl, sklearn, os
